In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install feature-engine

In [ ]:
#!pip install h2o

In [ ]:
#import h2o
#h2o.init()
#h2o.cluster().show_status()

In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import eli5
from warnings import filterwarnings
from sklearn.preprocessing import RobustScaler, MinMaxScaler,StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold,cross_val_score

from eli5.sklearn import PermutationImportance
from sklearn.preprocessing import FunctionTransformer, PowerTransformer
from imblearn.over_sampling import SMOTE,RandomOverSampler,ADASYN,KMeansSMOTE,SMOTENC,BorderlineSMOTE,SVMSMOTE
from imblearn.under_sampling import NearMiss,RandomUnderSampler,TomekLinks,OneSidedSelection
from imblearn.under_sampling import CondensedNearestNeighbour,RepeatedEditedNearestNeighbours
from imblearn.under_sampling import  ClusterCentroids,InstanceHardnessThreshold
from imblearn.ensemble import BalancedBaggingClassifier,BalancedRandomForestClassifier,EasyEnsembleClassifier
from collections import Counter
from sklearn.ensemble import VotingClassifier,RandomForestClassifier,GradientBoostingClassifier,StackingClassifier
from sklearn.cluster import KMeans
from scipy.stats import boxcox
from scipy import stats
import scipy
from feature_engine.encoding import WoEEncoder,MeanEncoder,PRatioEncoder
from feature_engine.encoding import DecisionTreeEncoder,CountFrequencyEncoder,OrdinalEncoder
from feature_engine.selection import DropConstantFeatures,SmartCorrelatedSelection
from feature_engine.creation import MathematicalCombination,CombineWithReferenceFeature
from sklearn.tree import DecisionTreeClassifier
import featuretools as ft
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import missingno as msn
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.ensemble import IsolationForest

filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

In [ ]:
submission=pd.read_csv("/kaggle/input/turkiye-is-bankas-machine-learning-challenge-3/sample_submission.csv")
monthly_exp=pd.read_csv("/kaggle/input/turkiye-is-bankas-machine-learning-challenge-3/monthly_expenditures.csv")
train=pd.read_csv("/kaggle/input/turkiye-is-bankas-machine-learning-challenge-3/train.csv")
test=pd.read_csv("/kaggle/input/turkiye-is-bankas-machine-learning-challenge-3/test.csv")

In [ ]:
train=train.drop([29804,48736])

In [ ]:
train_musteri=train.musteri
test_musteri=test.musteri

In [ ]:
full_data=pd.concat([train,test])
full_data_musteri=full_data.musteri

In [ ]:
monthly_exp.tarih=monthly_exp.tarih.astype(str)

In [ ]:
monthly_exp["date"]=[monthly_exp.tarih[i][:4]+"-"+monthly_exp.tarih[i][4:] for i in range(len(monthly_exp.tarih))]

In [ ]:
monthly_exp["date"]=[monthly_exp.date[i][:7]+"-"+monthly_exp.date[i][7:] for i in range(len(monthly_exp.date))]

In [ ]:
monthly_exp.date=pd.to_datetime(monthly_exp.date)
monthly_exp["ay"]=monthly_exp.date.dt.month

In [ ]:
merged_data=pd.concat([full_data,monthly_exp])

In [ ]:
merged_data_new=pd.merge(merged_data,full_data,on=["musteri"]).drop(["yas_x","kidem_suresi_x",
                                                     "egitim_x","is_durumu_x",
                                                    "meslek_grubu_x","tarih_x",
                                                    "target_x"],axis=1).rename({"yas_y":"yas",
                                                                               "kidem_suresi_y":"kidem_suresi",
                                                                               "egitim_y":"egitim",
                                                                               "is_durumu_y":"is_durumu",
                                                                               "meslek_grubu_y":"meslek_grubu",
                                                                               "tarih_y":"tarih",
                                                                               "target_y":"target"},axis=1)
merged_data_new=merged_data_new.dropna(subset=["sektor","islem_adedi",
                                               "aylik_toplam_tutar","date","ay"])
merged_data_new.head()

In [ ]:
#sns.distplot(merged_data_new.kidem_suresi);

In [ ]:
merged_data_new.kidem_suresi.describe().to_frame().T

In [ ]:
#merged_data_new["kidem_ordinal"]=np.nan
#merged_data_new.loc[(merged_data_new.kidem_suresi<=33),"kidem_ordinal"]=1
#merged_data_new.loc[(merged_data_new.kidem_suresi>33) &(merged_data_new.kidem_suresi<=71),"kidem_ordinal"]=2
#merged_data_new.loc[(merged_data_new.kidem_suresi>71) &(merged_data_new.kidem_suresi<=131),"kidem_ordinal"]=3
#merged_data_new.loc[(merged_data_new.kidem_suresi>131)&(merged_data_new.kidem_suresi<=300),"kidem_ordinal"]=4
#merged_data_new.loc[(merged_data_new.kidem_suresi>300),"kidem_ordinal"]=5

In [ ]:
#merged_data_new["count_0"]=np.nan
#merged_data_new.loc[merged_data_new.aylik_toplam_tutar==0,"count_0"]=0
#merged_data_new.count_0=merged_data_new.count_0.fillna(1)


In [ ]:
merged_data_new.aylik_toplam_tutar.max()

In [ ]:
merged_data_new.aylik_toplam_tutar.describe().to_frame().T

In [ ]:
merged_data_new=merged_data_new.drop([230981])

In [ ]:
merged_data_new["yas_grubu"]=pd.cut(merged_data_new.yas,bins=[17,19,24,29,34,39,44,50],
                        labels=["grup1","grup2","grup3","grup4","grup5","grup6","grup7"])



In [ ]:
merged_data_new.loc[merged_data_new.islem_adedi<=0,"islem_adedi"]=-1*(merged_data_new.loc[merged_data_new.islem_adedi<=0,"islem_adedi"])

In [ ]:
count_map_=merged_data_new.sektor.value_counts().to_dict()
merged_data_new["count_sektor"]=merged_data_new.sektor.map(count_map_)

In [ ]:
count_map=merged_data_new.musteri.value_counts().to_dict()
merged_data_new["count_customer"]=merged_data_new.musteri.map(count_map)

In [ ]:
islem_adet_toplam=merged_data_new.groupby(["musteri"])["islem_adedi"].sum()
merged_data_new=pd.merge(merged_data_new,islem_adet_toplam,how="inner",on=["musteri"]).rename({"islem_adedi_x":"islem_adedi",
                                                                            "islem_adedi_y":"islem_adet_toplam"},
                                                                           axis=1).sort_values(by="date")

In [ ]:
mus_tutar=merged_data_new.groupby(["musteri"])["aylik_toplam_tutar"].sum()
merged_data_new=pd.merge(merged_data_new,mus_tutar,how="inner",on=["musteri"]).rename({"aylik_toplam_tutar_x":"aylik_toplam_tutar",
                                                                          "aylik_toplam_tutar_y":"mus_tutar"},
                                                                           axis=1).sort_values(by="date")

#mus_std_tutar=merged_data_new.groupby(["musteri"])["aylik_toplam_tutar"].std()
#merged_data_new=pd.merge(merged_data_new,mus_std_tutar,how="inner",on=["musteri"]).rename({"aylik_toplam_tutar_x":"aylik_toplam_tutar",
 #                                                                           "aylik_toplam_tutar_y":"mus_std_tutar"},
  #                                                                         axis=1).sort_values(by="date")

In [ ]:
count_map=merged_data_new.loc[merged_data_new.sektor=="KUYUMCU","musteri"].value_counts().to_dict()

merged_data_new["count_KUYUMCU"]=merged_data_new.musteri.map(count_map)
merged_data_new.count_KUYUMCU=merged_data_new.count_KUYUMCU.fillna(0)

In [ ]:
count_map=merged_data_new.loc[merged_data_new.sektor=="DIJITAL_URUNLR","musteri"].value_counts().to_dict()

merged_data_new["count_DIGITAL"]=merged_data_new.musteri.map(count_map)
merged_data_new.count_DIGITAL=merged_data_new.count_DIGITAL.fillna(0)


In [ ]:
count_map=merged_data_new.loc[merged_data_new.sektor=="BAHCE_CICEKCLK","musteri"].value_counts().to_dict()

merged_data_new["count_CICEK"]=merged_data_new.musteri.map(count_map)
merged_data_new.count_CICEK=merged_data_new.count_CICEK.fillna(0)


In [ ]:
count_map=merged_data_new.loc[merged_data_new.sektor=="TASIMACILIK","musteri"].value_counts().to_dict()

merged_data_new["count_TASIMA"]=merged_data_new.musteri.map(count_map)
merged_data_new.count_TASIMA=merged_data_new.count_TASIMA.fillna(0)



In [ ]:
count_map=merged_data_new.loc[merged_data_new.sektor=="BENZIN_YAKIT","musteri"].value_counts().to_dict()

merged_data_new["count_BENZIN"]=merged_data_new.musteri.map(count_map)
merged_data_new.count_BENZIN=merged_data_new.count_BENZIN.fillna(0)

In [ ]:
count_map=merged_data_new.loc[merged_data_new.sektor=="GIYIM_AKSESUAR","musteri"].value_counts().to_dict()

merged_data_new["count_GIYIM"]=merged_data_new.musteri.map(count_map)
merged_data_new.count_GIYIM=merged_data_new.count_GIYIM.fillna(0)

In [ ]:
count_map=merged_data_new.loc[merged_data_new.sektor=="ELKT_ESYA_BILG","musteri"].value_counts().to_dict()

merged_data_new["count_ELK"]=merged_data_new.musteri.map(count_map)
merged_data_new.count_ELK=merged_data_new.count_ELK.fillna(0)

In [ ]:
count_map=merged_data_new.loc[merged_data_new.sektor=="RESTORAN_CATER","musteri"].value_counts().to_dict()

merged_data_new["count_REST"]=merged_data_new.musteri.map(count_map)
merged_data_new.count_REST=merged_data_new.count_REST.fillna(0)

In [ ]:
count_map=merged_data_new.loc[merged_data_new.sektor=="ICKILI_YERLER","musteri"].value_counts().to_dict()

merged_data_new["count_ICKILI_YERLER"]=merged_data_new.musteri.map(count_map)
merged_data_new.count_ICKILI_YERLER=merged_data_new.count_ICKILI_YERLER.fillna(0)

In [ ]:
count_map=merged_data_new.loc[merged_data_new.sektor=="EGL_SPOR_HOBI","musteri"].value_counts().to_dict()

merged_data_new["count_EGL_SPOR_HOBI"]=merged_data_new.musteri.map(count_map)
merged_data_new.count_EGL_SPOR_HOBI=merged_data_new.count_EGL_SPOR_HOBI.fillna(0)


In [ ]:

count_map=merged_data_new.loc[merged_data_new.sektor=="KLP_DERNK_SOSY","musteri"].value_counts().to_dict()

merged_data_new["count_KLP_DERNK_SOSY"]=merged_data_new.musteri.map(count_map)
merged_data_new.count_KLP_DERNK_SOSY=merged_data_new.count_KLP_DERNK_SOSY.fillna(0)

In [ ]:

count_map=merged_data_new.loc[merged_data_new.sektor=="SAGLIK_HIZMTLR","musteri"].value_counts().to_dict()

merged_data_new["count_SAGLIK_HIZMTLR"]=merged_data_new.musteri.map(count_map)
merged_data_new.count_SAGLIK_HIZMTLR=merged_data_new.count_SAGLIK_HIZMTLR.fillna(0)

In [ ]:
def missing_indicator(df, variable):    
    return np.where(df[variable].isnull(), 1, 0)

merged_data_new["meslek_grubu"+'_NA'] = missing_indicator(merged_data_new,"meslek_grubu")
#merged_data_new["egitim"+'_NA'] = missing_indicator(merged_data_new, "egitim")
#merged_data_new["is_durumu"+"_NA"]=missing_indicator(merged_data_new,"is_durumu")

In [ ]:

#merged_data_new.meslek_grubu=merged_data_new.meslek_grubu.fillna("bilinmiyor")
#merged_data_new.egitim=merged_data_new.egitim.fillna("bilinmiyor")
#merged_data_new.is_durumu=merged_data_new.is_durumu.fillna("bilinmiyor")


merged_data_new.meslek_grubu=merged_data_new.meslek_grubu.fillna("44af6626d6")
merged_data_new.egitim=merged_data_new.egitim.fillna("7e6640bfe0")
merged_data_new.is_durumu=merged_data_new.is_durumu.fillna("991c4998fb")




In [ ]:
mean_KUYUMCU=merged_data_new.loc[merged_data_new.sektor=="KUYUMCU",["musteri","aylik_toplam_tutar"]].groupby("musteri")["aylik_toplam_tutar"].sum().to_dict()
merged_data_new["mean_KUYUMCU"]=merged_data_new.musteri.map(mean_KUYUMCU)
merged_data_new.mean_KUYUMCU=merged_data_new.mean_KUYUMCU.fillna(0)

In [ ]:
mean_GIYIM=merged_data_new.loc[merged_data_new.sektor=="GIYIM_AKSESUAR",["musteri","aylik_toplam_tutar"]].groupby("musteri")["aylik_toplam_tutar"].sum().to_dict()
merged_data_new["mean_GIYIM"]=merged_data_new.musteri.map(mean_GIYIM)
merged_data_new.mean_GIYIM=merged_data_new.mean_GIYIM.fillna(0)

In [ ]:
mean_DIGITAL=merged_data_new.loc[merged_data_new.sektor=="DIJITAL_URUNLR",["musteri","aylik_toplam_tutar"]].groupby("musteri")["aylik_toplam_tutar"].sum().to_dict()
merged_data_new["mean_DIGITAL"]=merged_data_new.musteri.map(mean_DIGITAL)
merged_data_new.mean_DIGITAL=merged_data_new.mean_DIGITAL.fillna(0)

In [ ]:
mean_TASIMA=merged_data_new.loc[merged_data_new.sektor=="TASIMACILIK",["musteri","aylik_toplam_tutar"]].groupby("musteri")["aylik_toplam_tutar"].sum().to_dict()
merged_data_new["mean_TASIMA"]=merged_data_new.musteri.map(mean_TASIMA)
merged_data_new.mean_TASIMA=merged_data_new.mean_TASIMA.fillna(0)

In [ ]:
mean_BAHCE=merged_data_new.loc[merged_data_new.sektor=="BAHCE_CICEKCLK",["musteri","aylik_toplam_tutar"]].groupby("musteri")["aylik_toplam_tutar"].sum().to_dict()
merged_data_new["mean_BAHCE"]=merged_data_new.musteri.map(mean_BAHCE)
merged_data_new.mean_BAHCE=merged_data_new.mean_BAHCE.fillna(0)

In [ ]:
mean_ELEK=merged_data_new.loc[merged_data_new.sektor=="ELKT_ESYA_BILG",["musteri","aylik_toplam_tutar"]].groupby("musteri")["aylik_toplam_tutar"].sum().to_dict()
merged_data_new["mean_ELEK"]=merged_data_new.musteri.map(mean_ELEK)
merged_data_new.mean_ELEK=merged_data_new.mean_ELEK.fillna(0)

In [ ]:
mean_BENZIN=merged_data_new.loc[merged_data_new.sektor=="BENZIN_YAKIT",["musteri","aylik_toplam_tutar"]].groupby("musteri")["aylik_toplam_tutar"].sum().to_dict()
merged_data_new["mean_BENZIN"]=merged_data_new.musteri.map(mean_BENZIN)
merged_data_new.mean_BENZIN=merged_data_new.mean_BENZIN.fillna(0)

In [ ]:
mean_REST=merged_data_new.loc[merged_data_new.sektor=="RESTORAN_CATER",["musteri","aylik_toplam_tutar"]].groupby("musteri")["aylik_toplam_tutar"].sum().to_dict()
merged_data_new["mean_REST"]=merged_data_new.musteri.map(mean_REST)
merged_data_new.mean_REST=merged_data_new.mean_REST.fillna(0)

In [ ]:
mean_ICKI=merged_data_new.loc[merged_data_new.sektor=="ICKILI_YERLER",["musteri","aylik_toplam_tutar"]].groupby("musteri")["aylik_toplam_tutar"].sum().to_dict()
merged_data_new["mean_ICKI"]=merged_data_new.musteri.map(mean_ICKI)
merged_data_new.mean_ICKI=merged_data_new.mean_ICKI.fillna(0)

In [ ]:
mean_DERNEK=merged_data_new.loc[merged_data_new.sektor=="KLP_DERNK_SOSY",["musteri","aylik_toplam_tutar"]].groupby("musteri")["aylik_toplam_tutar"].sum().to_dict()
merged_data_new["mean_DERNEK"]=merged_data_new.musteri.map(mean_DERNEK)
merged_data_new.mean_DERNEK=merged_data_new.mean_DERNEK.fillna(0)

In [ ]:
merged_data_new.head()

# -----------------------------------------------------------------------

In [ ]:
unique_data=merged_data_new.drop_duplicates(subset="musteri")

In [ ]:
full_data_musteri=pd.DataFrame(full_data_musteri)

In [ ]:
full=full_data_musteri.join(unique_data.set_index("musteri"),on="musteri").drop(["islem_adedi",
                                                                                 "aylik_toplam_tutar","ay","sektor"],axis=1)
full.head() #sektor eklicen

In [ ]:
full.loc[full.kidem_suresi<0,"kidem_suresi"]=0

In [ ]:
full.head()

In [ ]:
full.isnull().sum()

In [ ]:
full.shape

In [ ]:
train_new=full.iloc[:59998]
test_new=full.iloc[59998:]

In [ ]:
train_new["log_kidem_suresi"]=np.sqrt(train_new.kidem_suresi)
test_new["log_kidem_suresi"]=np.sqrt(test_new.kidem_suresi)

In [ ]:
#plt.figure(figsize=(20,10));
#sns.heatmap(train_new.corr().abs(),annot=True,cmap="coolwarm");

In [ ]:
X_train=train_new.drop(["musteri","date","tarih","target","kidem_suresi","yas_grubu"],axis=1)

#"grup_aylik_mean"
X_test=test_new.drop(["musteri","date","tarih","target","kidem_suresi","yas_grubu"],axis=1)
y_train=train_new["target"]
X_train.shape,X_test.shape,y_train.shape

In [ ]:
ratio_enc =PRatioEncoder(
    variables=['meslek_grubu',"egitim","is_durumu"])

#PRatioEncoder
#MeanEncoder


In [ ]:
ratio_enc.fit(X_train, y_train)
X_train = ratio_enc.transform(X_train)
X_test = ratio_enc.transform(X_test)

In [ ]:
X_train_encoded=pd.get_dummies(X_train)
X_test_encoded=pd.get_dummies(X_test)
X_train_encoded.shape,X_test_encoded.shape

In [ ]:
X_train_encoded.shape,y_train.shape

In [ ]:
scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train_encoded)
X_test_scaled=scaler.transform(X_test_encoded)

In [ ]:
def weight_ratio(y_train):
    num = y_train.value_counts()
    return num[0]/num[1]
ratio=weight_ratio(y_train)
ratio

In [ ]:

model=LGBMClassifier(learning_rate=0.008,
                 max_depth=3,
                     n_estimators=2050,
                     reg_lambda=45,
                     boosting_type="goss", #gbdt goss dart               
                     min_child_weight=0.001,
                     min_child_samples=23,
                     objective="binary",
                     num_leaves=31,
                     importance_type='split' #split gain                                       
                  
                   )
#scale_pos_weight=ratio

In [ ]:
smt = RandomOverSampler(sampling_strategy="minority") #(sampling_strategy="minority")
#smt=KMeansSMOTE(sampling_strategy="auto",k_neighbors=3,kmeans_estimator=KMeans(n_clusters=3),cluster_balance_threshold=0.05,density_exponent="auto")
#smt=SVMSMOTE(sampling_strategy="minority",svm_estimator=SVC(kernel="linear"))
X_train_scaled, y_train = smt.fit_sample(X_train_scaled, y_train)




In [ ]:
X_train_scaled.shape,y_train.shape

In [ ]:
sns.countplot(y_train);

In [ ]:
model_fitted=model.fit(X_train_scaled,y_train)
pred=model_fitted.predict_proba(X_test_scaled)

In [ ]:
pred=pred[:,1]

In [ ]:
model.score(X_train_scaled,y_train)

In [ ]:
submission.target=pred

In [ ]:
submission.head(50)

In [ ]:
train.target.value_counts()/len(train)

In [ ]:
pd.DataFrame(pred,columns=["pred"]).pred.value_counts()/len(pred)

In [ ]:
sns.countplot(pred);

In [ ]:
Importance = pd.DataFrame({"Importance": model.feature_importances_},
                         index = X_train_encoded.columns)
Importance.sort_values(by = "Importance", 
                       axis = 0, 
                       ascending = True).plot(kind ="barh", color = "r",figsize=(20,10));

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)

scores = cross_val_score(model, X_train_scaled, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

print('Mean ROC AUC: %.5f' % np.mean(scores))

In [ ]:
scores

In [ ]:
submission_csv=submission.to_csv("submission.csv",index=False)
from IPython.display import FileLink
FileLink(r'submission.csv')